## 데이터 가공

#### 지시사항

1. data/flight-data/json 경로의 json 파일을 읽으세요. 여러 파일을 한꺼번에 읽을땐 * 표시를 하면 한꺼번에 읽을 수 있습니다 data/flight-data/csv 경로의 csv 파일을 읽으세요. header=True 옵션을 주어야 합니다.

2. select와 selectExpr을 사용하여 ‘count’ 컬럼의 평균을 각각 구하세요.

3. isin을 사용하여 ‘DEST_COUNTRY_NAME’ 컬럼의 (‘United States’,’Germany’,’Russia’) 국가들만 필터링하고 ‘count’의 합계를 구하세요.

4. 'DEST_COUNTRY_NAME’ 에 따라(groupby) ‘count’의 평균을 구하세요.

5. crosstab을 사용하여 ‘DEST_COUNTRY_NAME’ ,’ORIGIN_COUNTRY_NAME’의 교차표를 구하세요.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()


# JSON 파일 읽기(json파일없음)
# df_json = spark.read.json("data/flight-data/json/*")
# df_json.show()

# CSV 파일 읽기
df_csv = spark.read.csv("data/flight-data/*", header=True)
df_csv.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [4]:
# select를 활용하여 평균 구하기
df_csv.select(avg('count')).show()

+------------------+
|        avg(count)|
+------------------+
|1751.0575635876842|
+------------------+



In [5]:
# selectExpr을 활용하여 평균 구하기
df_csv.selectExpr('avg(count)').show()

+--------------------------+
|avg(CAST(count AS DOUBLE))|
+--------------------------+
|        1751.0575635876842|
+--------------------------+



In [6]:
# isin을 사용하여 필터링하고, 합계 구하기
df_csv.where(col('DEST_COUNTRY_NAME').isin(['United States', 'Germany', 'Russia'])).select(sum('count')).show()

+----------+
|sum(count)|
+----------+
| 1193366.0|
+----------+



In [7]:
# 그룹별 평균 구하기
df_csv.groupby('DEST_COUNTRY_NAME').agg(avg('count')).show()

+-----------------+------------------+
|DEST_COUNTRY_NAME|        avg(count)|
+-----------------+------------------+
|             Chad|               1.0|
|         Anguilla|32.333333333333336|
|           Russia|194.33333333333334|
|         Paraguay|              75.0|
|          Senegal|34.333333333333336|
|           Sweden|              94.0|
|         Kiribati|              27.0|
|           Guyana|              50.0|
|      Philippines|134.66666666666666|
|         Djibouti|               1.0|
|         Malaysia|               2.0|
|        Singapore|              12.0|
|             Fiji|27.666666666666668|
|           Turkey|109.33333333333333|
|           Malawi|               1.0|
|             Iraq|               1.0|
|          Germany|1427.3333333333333|
|           Jordan|              61.0|
|           Rwanda|               1.0|
|            Palau|34.333333333333336|
+-----------------+------------------+
only showing top 20 rows



In [8]:
# 교차표 만들기
df_csv.crosstab('DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME').show()

+-------------------------------------+------+--------+-------------------+---------+-----+---------+-------+----------+-------+--------+-------+------+-------+-------+---------------------------------+------+----------------------+--------+--------+------+----------+--------------+-----+-----+--------+------------------+------------+----------+-------+----+-------+------+--------------+-------+--------+------------------+-------+-----+-----------+-------+--------+------------------------------+----+-------+------+-------------+----------------+-------+-------+-----+---------+------+---------+-------+----------+---------+------+-----+--------+---------+-------+-------+-----+---------+-------+------+-----+-------+-----+------+----------+--------+------+------+-------+---------+----------+--------+----+-----+----------------+----------+------+----------+-------+-------+-----------+-----------+---------+-----+-------+------+--------+-----+------+----------------+--------+----+-----------+